In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as sia

In [2]:
headers = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
   }
# f_id = "0x3bae1598dd57bf07:0xe4cf400d233ae09"
f_id = "0x3bae1405dbb04b89:0x6ba4a1a96e5ed004"

response = requests.get(f"https://www.google.com/async/reviewDialog?hl=en_us&async=feature_id:{f_id},next_page_token:,sort_by:qualityScore,start_index:,associated_topic:,_fmt:pc", headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

user = []
location_info = {}
data_id = ''
token = ''

def create_date(date):
    num = date.split(' ')[0]
    duration = date.split(' ')[1]
    if duration in ['second','seconds','minute','minutes','hour','hours','day','days','week','weeks']:
        currentTimeDate = datetime.now()
    elif duration == 'month':
        currentTimeDate = datetime.now() - relativedelta(months=1)
    elif duration == 'months':
        currentTimeDate = datetime.now() - relativedelta(months=int(num))
    elif duration == 'year':
        currentTimeDate = datetime.now() - relativedelta(years=1)
    elif duration == 'years':
        currentTimeDate = datetime.now() - relativedelta(years=int(num))
    newTime = currentTimeDate.strftime('%Y-%m-%d')
    return newTime

for el in soup.select('.lcorif'):
    data_id = soup.select_one('.loris')['data-fid']
    token = soup.select_one('  ')['data-next-page-token']
    location_info = {
              'title': soup.select_one('.P5Bobd').text.strip(),
               'address': soup.select_one('.T6pBCe').text.strip(),
               'avgRating': soup.select_one('span.Aq14fc').text.strip(),
               'totalReviews': soup.select_one('span.z5jxId').text.strip()
           }
    
    for el in soup.select('.gws-localreviews__google-review'):
           user.append({
               'name': el.select_one('.TSUbDb').text.strip(),
               'rating': el.select_one('.BgXiYe .lTi8oc')['aria-label'],
               'review': el.select_one('.Jtu6Td').text.strip(),
               'date': create_date(el.select_one('.PuaHbe').text.strip()),
           })
print(location_info)
def token_check(token):
    f_id = "ChIJiUuw2wUUrjsRBNBebqmhpGs"
    headers = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
   }
    response = requests.get(f"https://www.google.com/async/reviewDialog?hl=en_us&async=feature_id:{f_id},next_page_token:{token},sort_by:qualityScore,start_index:,associated_topic:,_fmt:pc", headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    user = []
    for el in soup.select('.lcorif'):
       data_id = soup.select_one('.loris')['data-fid']
       token = soup.select_one('.gws-localreviews__general-reviews-block')['data-next-page-token']
    for el in soup.select('.gws-localreviews__google-review'):
           user.append({
               'name': el.select_one('.TSUbDb').text.strip(),
               'rating': el.select_one('.BgXiYe .lTi8oc')['aria-label'],
               'review': el.select_one('.Jtu6Td').text.strip(),
               'date': create_date(el.select_one('.PuaHbe').text.strip()),
           })
    res = {
            'token':token,
            'user': user
           }
    return res
print(f'1. {token}  n')
n = 2
while True:
    res =  token_check(token)
    new_token = res['token']
    token = new_token
    user.extend(res['user'])
    print(f'{n}. {token}  {len(user)}')
    n = n+1
    if not token:
        False
        break

{}
1.   n
2.   0


In [3]:
len(user)

0

In [1]:
print(location_info)

NameError: name 'location_info' is not defined

In [3]:
df = pd.read_excel ('Rosetta.xlsx')

In [4]:
df.columns

Index(['name', 'rating', 'review'], dtype='object')

In [5]:
df

,name,rating,review
0,Dileep Kumar,5/5,NaN
1,Ayush,5/5,Mini Maldives kind of feeling. Very nice exper...
2,Vishal Kulkarni,5/5,NaN
3,Ekta Pinto,5/5,NaN
4,A S Niharika,5/5,NaN
...,...,...,...
1135,Noyel Royes,4/5,Good
1136,anjali prabhakaran,5/5,The property is a fabulous one. The staff is s...
1137,Amshu,5/5,"Everything in our 2 nights, 3days stay was ver..."
1138,senthil pandurangam,5/5,Excellent property for hygeine.. stay was amaz...


In [6]:
df.fillna('',inplace=True)

In [9]:
# name = location_info['title']
# name = name.replace(' ','_')
# name

In [7]:
name2 = ""
df.to_csv('Srinidhi_sentiments.csv',index = False)

In [15]:

def sentiment_scores(x):
    sentence = x['review']
    rating = eval(x['rating'])
    # print(type(rating))
    
    sid_obj = sia()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    print(sentiment_dict)
    pos = sentiment_dict['pos']*100
    neg = sentiment_dict['neg']*100
    neu = sentiment_dict['neu']*100

    if pos > neg:
        max = pos
        sentiment = 'Positive'
    else:
        max = neg
        sentiment = 'Negative'
    if max < neu:
        max = neu
        sentiment = 'Neutral'         

    if sentiment == 'Negative' and rating <2:
        print("this is rating:",rating)
        sentiment = 'Extreme'
    if sentence == '':
        sentiment = 'Neutral'
    print(sentiment)
    return sentiment

  

In [9]:
#df['rating'] = df['rating'].apply(lambda x : eval(x.split(' ')[0]))

In [23]:
df['rating'] = df['rating'].apply(lambda x : (x.split('/')[0]))
print(df['rating'])

0       5
1       5
2       5
3       5
4       5
       ..
1135    4
1136    5
1137    5
1138    5
1139    1
Name: rating, Length: 1140, dtype: object


In [16]:
df['sentiment'] = df.apply(sentiment_scores,axis=1)

{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.719, 'pos': 0.281, 'compound': 0.5106}
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.793, 'pos': 0.207, 'compound': 0.8873}
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.091, 'neu': 0.584, 'pos': 0.325, 'compound': 0.5574}
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rating: 1.0
Neutral
{'neg': 0.0, 'neu': 0.868, 'pos': 0.132, 'compound': 0.6249}
Neutral
{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
this is rati

In [18]:
set(list(df['sentiment']))

{'Extreme', 'Neutral', 'Positive'}

In [ ]:
# name = location_info['title']
# name = name.replace(' ','_')
# name

In [21]:
df.to_excel('rosetta_sen.xlsx',index = False)

In [22]:
df

,name,rating,review,sentiment
0,Dileep Kumar,5/5,,Neutral
1,Ayush,5/5,Mini Maldives kind of feeling. Very nice exper...,Neutral
2,Vishal Kulkarni,5/5,,Neutral
3,Ekta Pinto,5/5,,Neutral
4,A S Niharika,5/5,,Neutral
...,...,...,...,...
1135,Noyel Royes,4/5,Good,Positive
1136,anjali prabhakaran,5/5,The property is a fabulous one. The staff is s...,Neutral
1137,Amshu,5/5,"Everything in our 2 nights, 3days stay was ver...",Neutral
1138,senthil pandurangam,5/5,Excellent property for hygeine.. stay was amaz...,Neutral


In [ ]:
from datetime import datetime
x =  "2023-06-03 00:00:00+00:00" 
